<a href="https://colab.research.google.com/github/joker2017/Calculator/blob/master/Mobilenet_v2_SC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
"""Implementation of Mobilenet V2.

Architecture: https://arxiv.org/abs/1801.04381

The base model gives 72.2% accuracy on ImageNet, with 300MMadds,
3.4 M parameters.
"""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import copy
import functools

import tensorflow as tf

from nets.mobilenet import conv_blocks as ops
from nets.mobilenet import mobilenet as lib

slim = tf.contrib.slim
op = lib.op

expand_input = ops.expand_input_by_factor

# pyformat: disable
# Architecture: https://arxiv.org/abs/1801.04381
V2_DEF = dict(
    defaults={
        # Примечание: эти параметры batch_norm влияют на архитектуру
         # именно поэтому они здесь, а не в training_scope.
        (slim.batch_norm,): {'center': True, 'scale': True},
        (slim.conv2d, slim.fully_connected, slim.separable_conv2d): {
            'normalizer_fn': slim.batch_norm, 'activation_fn': tf.nn.relu6
        },
        (ops.expanded_conv,): {
            'expansion_size': expand_input(6),
            'split_expansion': 1,
            'normalizer_fn': slim.batch_norm,
            'residual': True
        },
        (slim.conv2d, slim.separable_conv2d): {'padding': 'SAME'}
    },
    spec=[
        op(slim.conv2d, stride=2, num_outputs=32, kernel_size=[3, 3]),
        op(ops.expanded_conv,
           expansion_size=expand_input(1, divisible_by=1),
           num_outputs=16),
        op(ops.expanded_conv, stride=2, num_outputs=24),
        op(ops.expanded_conv, stride=1, num_outputs=24),
        op(ops.expanded_conv, stride=2, num_outputs=32),
        op(ops.expanded_conv, stride=1, num_outputs=32),
        op(ops.expanded_conv, stride=1, num_outputs=32),
        op(ops.expanded_conv, stride=2, num_outputs=64),
        op(ops.expanded_conv, stride=1, num_outputs=64),
        op(ops.expanded_conv, stride=1, num_outputs=64),
        op(ops.expanded_conv, stride=1, num_outputs=64),
        op(ops.expanded_conv, stride=1, num_outputs=96),
        op(ops.expanded_conv, stride=1, num_outputs=96),
        op(ops.expanded_conv, stride=1, num_outputs=96),
        op(ops.expanded_conv, stride=2, num_outputs=160),
        op(ops.expanded_conv, stride=1, num_outputs=160),
        op(ops.expanded_conv, stride=1, num_outputs=160),
        op(ops.expanded_conv, stride=1, num_outputs=320),
        op(slim.conv2d, stride=1, kernel_size=[1, 1], num_outputs=1280)
    ],
)
# pyformat: enable

# **def mobilenet()**
Creates mobilenet V2 network.

Режим вывода создается по умолчанию. Для создания тренинга используйте training_scope ниже.

  with tf.contrib.slim.arg_scope(mobilenet_v2.training_scope()):
     logits, endpoints = mobilenet_v2.mobilenet(input_tensor)

> **Args:**
* *input_tensor:* входной тензор
* *num_classes:* количество классов
* *deep_multiplier:* множитель, применяемый для масштабирования количества каналов в каждом слое.Примечание: в документе это называется множителем глубины, но название сохранено для согласованности со строителем моделей Слима.
* *scope:* сфера действия оператора
* *conv_defs:* позволяет переопределить default def def.
* *finegrain_classification_mode:* при значении True модель будет сохранять последний слой большим даже для небольших множителей.
Followin https://arxiv.org/abs/1801.04381 предполагает, что это улучшает производительность для проблем типа ImageNet.* Примечание * игнорируется, если * *final_endpoint* заставляет построителя выйти раньше.
* *min_depth:* если предусмотрено, будет гарантировать, что все слои будут иметь столько каналов после применения умножителя глубины.
* *divisible_by:* Если это предусмотрено, убедитесь, что все слои # каналов будут делиться на это число.
* *Activation_FN:* функция активации для использования, по умолчанию tf.nn.relu6, если не указано.
* *kwargs:* передано непосредственно на mobilenet.mobilenet: Forextion_FN - какую функцию прогнозирования использовать. reuse-: повторное использование переменных (если reuse установлено в true, область действия должен быть дан). 

>*Return*: пара  logits/endpoints pair 
поднимает: ValueError: на недопустимых аргументах

In [0]:
@slim.add_arg_scope
def mobilenet(input_tensor,
              num_classes=1001,
              depth_multiplier=1.0,
              scope='MobilenetV2',
              conv_defs=None,
              finegrain_classification_mode=False,
              min_depth=None,
              divisible_by=None,
              activation_fn=None,
              **kwargs):

  if conv_defs is None:
    conv_defs = V2_DEF
  if 'multiplier' in kwargs:
    raise ValueError('mobilenetv2 doesn\'t support generic '
                     'multiplier parameter use "depth_multiplier" instead.')
  if finegrain_classification_mode:
    conv_defs = copy.deepcopy(conv_defs)
    if depth_multiplier < 1:
      conv_defs['spec'][-1].params['num_outputs'] /= depth_multiplier
  if activation_fn:
    conv_defs = copy.deepcopy(conv_defs)
    defaults = conv_defs['defaults']
    conv_defaults = (
        defaults[(slim.conv2d, slim.fully_connected, slim.separable_conv2d)])
    conv_defaults['activation_fn'] = activation_fn

  depth_args = {}
  # NB: do not set depth_args unless they are provided to avoid overriding
  # whatever default depth_multiplier might have thanks to arg_scope.
  if min_depth is not None:
    depth_args['min_depth'] = min_depth
  if divisible_by is not None:
    depth_args['divisible_by'] = divisible_by

  with slim.arg_scope((lib.depth_multiplier,), **depth_args):
    return lib.mobilenet(
        input_tensor,
        num_classes=num_classes,
        conv_defs=conv_defs,
        scope=scope,
        multiplier=depth_multiplier,
        **kwargs)

mobilenet.default_image_size = 224

In [0]:



def wrapped_partial(func, *args, **kwargs):
  partial_func = functools.partial(func, *args, **kwargs)
  functools.update_wrapper(partial_func, func)
  return partial_func


# Оболочки для mobilenet v2 с умножителями глубины. Обратите внимание, что
# 'finegrain_classification_mode' имеет значение True, что означает встраивание
# слой не будет уменьшен, если задан depth-multiplier <1,0.
mobilenet_v2_140 = wrapped_partial(mobilenet, depth_multiplier=1.4)
mobilenet_v2_050 = wrapped_partial(mobilenet, depth_multiplier=0.50,
                                   finegrain_classification_mode=True)
mobilenet_v2_035 = wrapped_partial(mobilenet, depth_multiplier=0.35,
                                   finegrain_classification_mode=True)


@slim.add_arg_scope
def mobilenet_base(input_tensor, depth_multiplier=1.0, **kwargs):
  """Creates base of the mobilenet (no pooling and no logits) ."""
  return mobilenet(input_tensor,
                   depth_multiplier=depth_multiplier,
                   base_only=True, **kwargs)


# **training_scope()**

Определяет область обучения MobilenetV2. 
>**Использование**: с помощью slim 

>*tf.contrib.slim.arg_scope (mobilenet_v2.training_scope ()):                                                                  
>logits, endpoints = mobilenet_v2.mobilenet (input_tensor)* 



> **Args:** 
* *kwargs:* передано в mobilenet.training_scope. Следующие параметры поддерживаются: 
 * weight_decay - снижение веса, используемое для регуляризации модели. 
 * stddev - стандартное отклонение для инициализации, если отрицательный использует xavier. 
 * dropout_keep_prob - вероятность для dropout
 * bn_decay - затухание для batch norm moving averages. 

> **Return:** 
* *Arg_scope* для использования в модели mobilenet v2. 

In [0]:


def training_scope(**kwargs):

  return lib.training_scope(**kwargs)


__all__ = ['training_scope', 'mobilenet_base', 'mobilenet', 'V2_DEF']